In [5]:
import rdflib
from rdflib import Graph, Literal, BNode, Namespace, RDF, URIRef
from rdflib.namespace import DC, FOAF
import spotlight
import urllib, json
import pandas as pd
from collections import Counter
import os

In [6]:
# load the graph
g = rdflib.Graph()
result = g.parse('kgraph215.ttl', format='turtle')

# setting self-namespaces 
cn = Namespace("http://ai.symptomchecking.org/condtion/")
dbpn = "http://dbpedia.org/resource/"
symptomOf = URIRef(cn.symptomOf)
linksTo = URIRef(cn.linkTo)
signOf = URIRef(cn.signOf)
anatomicOn = URIRef(cn.anatomicOn)
drug = URIRef(cn.drugTo)
cause = URIRef(cn.causes)
prob = URIRef(cn.hasProbability)

# define useful classes
disease = URIRef('http://dbpedia.org/ontology/Disease')
drug = URIRef('http://dbpedia.org/ontology/Drug')
symp = URIRef('http://dbpedia.org/resource/Category:Symptoms_and_signs')
anat = URIRef('http://dbpedia.org/ontology/AnatomicalStructure')

In [7]:
# import pandas as pd

In [8]:
def ranking_result(pdfs):
    sparl_Ranking(pdfs)
    ranking = pdfs.groupby('Disease')
    finalranking = ranking.sum().sort_values(by ='Probability', ascending=False )
    fr = finalranking.apply(calculate,axis=1)
    return fr.head(15)

In [9]:
def calculate(row):
    return row['Probability']/4

In [29]:
def sparl_Ranking(pdfs):
    for ix in s:
        sym_value = ix.replace(' ','_')
        queryi = dbpn+sym_value
        querystring = "SELECT DISTINCT ?d ?p WHERE {"
        querystring =  querystring+"{?d <"+prob+"> ?p} .}"
        sdg1=Graph()
        #print querystring
        gresult = sdg1.parse('dpbkg1.ttl', format='turtle')
        qres = sdg1.query(querystring)
        i=0
        for row in qres:
            d = str(row.asdict()['d'].toPython())
            p = float(str(row.asdict()['p'].toPython()).split(':')[1])
            pdfs.loc[i]=[d]+[p]
            i=i+1
    return pdfs

In [11]:
def sparl_signlist(s,sdg):
    for ix in s:
        sym_value = ix.replace(' ','_')
        queryi = dbpn+sym_value
        querystring = "CONSTRUCT {?d <"+cause+"> <"+queryi+">} WHERE {"
        querystring =  querystring+"{<"+queryi+"> <"+signOf+"> ?d} UNION {<"+queryi+"> <"+symptomOf+"> ?d} .}"
        #print querystring
        qres = g.query(querystring)
        probv=0
        if (len(qres)!=0):
            probv = Literal(sym_value+":"+str(1.0/float(len(qres))))
            #print probv
        for row in qres:
            #print row[0]
            sdg.add(row)
            sdg.add((row[0], prob, probv))
    return "1"

In [12]:
def sparl_signmap(s,sdg):
    for ix in s:
        sym_value = ix.replace(' ','_')
        queryi = dbpn+sym_value
        querystring = "SELECT DISTINCT ?d WHERE {"
        querystring =  querystring+"{<"+queryi+"> <"+signOf+"> ?d} UNION {<"+queryi+"> <"+symptomOf+"> ?d} .}"
        #print querystring
        qres = g.query(querystring)
        for row in qres:
            d = str(row.asdict()['d'].toPython())
            querylinkTo = "CONSTRUCT {?l <"+linksTo+"> <"+d+"> }  WHERE {<"+d+"> <"+linksTo+"> ?l .}"
            qlr = g.query(querylinkTo)
            probv=0
            if (len(qlr)!=0):
                probv = Literal(sym_value+":"+str(1.0/(float(len(qlr))*float(len(qres)))))
                #print probv
            for links in qlr:
                sdg.add(links)
                #print '11'
                i_term = URIRef(queryi)
                sdg.add((links[0], cause, i_term))
                sdg.add((links[0], prob, probv))
                #print '12'
    return "1"

In [13]:
def sparl_age_gender_dislist(s, sdg):
    j=0
    querystring = "SELECT DISTINCT ?d WHERE {"
    for xqi in s:
        if (j<len(s)-1):
            querystring = querystring + "{\
            ?d <http://purl.org/dc/elements/1.1/abstract> ?a . FILTER regex(str(?a), '"+xqi+"')} UNION {\
            ?d <http://purl.org/dc/elements/1.1/abstract> ?a . FILTER regex(str(?a), '"+xqi.lower()+"')} UNION"
        else:
            querystring = querystring + "{\
            ?d <http://purl.org/dc/elements/1.1/abstract> ?a . FILTER regex(str(?a), '"+xqi+"')} UNION {\
            ?d <http://purl.org/dc/elements/1.1/abstract> ?a . FILTER regex(str(?a), '"+xqi.lower()+"')}"
        j=j+1
    querystring = querystring + "}"
    #print (querystring)
    qres = g.query(querystring)
    for row in qres:
        d = str(row.asdict()['d'].toPython())
        querylinkTo = "CONSTRUCT {<"+d+"> <"+prob+"> ?l .} WHERE {<"+d+"> <"+prob+"> ?l .}"
        qlr = sdg.query(querylinkTo)
        if (len(qlr)!=0):
            #probv = 0.5*(1/(float(len(qlr))))
            #print probv
            probv = 0.5
        for links in qlr:
            #sdg.add(links)
            #linkv = links[2].split(':')[1]
            #probvl = Literal("aggregation:"+str(float(linkv)+probv))
            probvl = Literal("ageOrGender:"+str(probv))
            #sdg.remove((links[0],prob, None))
            sdg.add((links[0], prob, probvl))
    return '1'

In [14]:
def sparl_aidiseaselist(s, sdg):
    querystring = "SELECT DISTINCT ?d WHERE {"
    j=0
    for ix in s:
        queryi = ix.replace(' ','_')
        if(j<len(s)-1):
            querystring = querystring + "{?d <"+anatomicOn+"> ?a . FILTER regex(str(?a), '"+queryi+"')} UNION {?d <"+anatomicOn+"> ?a . FILTER regex(str(?a), '"+queryi.lower()+"')} UNION "
        else:
            querystring = querystring + "{?d <"+anatomicOn+"> ?a . FILTER regex(str(?a), '"+queryi+"')} UNION {?d <"+anatomicOn+"> ?a . FILTER regex(str(?a), '"+queryi.lower()+"')} ."
        j=j+1
    querystring = querystring+"}"
    qres = g.query(querystring)
    for row in qres:
        d = str(row.asdict()['d'].toPython())
        querylinkTo = "CONSTRUCT {<"+d+"> <"+prob+"> ?l .} WHERE {<"+d+"> <"+prob+"> ?l .}"
        qlr = sdg.query(querylinkTo)
        if (len(qlr)!=0 and len(qres)!=0):
            lens = len(qlr)+len(qres)
            probv = 1.00/float(lens)
            #print probv
        for links in qlr:
            #sdg.add(links)
            #linkv = links[2].split(':')[1]
            #probvl = Literal("aggregation:"+str(float(linkv)+probv))
            probvl = Literal("position:"+str(probv))
            sdg.add((links[0], prob, probvl))
    return "1"

In [15]:
def dpbkg_generation(sdg):
    filepath='dpbkg1.ttl'
    sdg.serialize(destination=filepath, format='turtle')
    return sdg

In [16]:
s = [' Baby',' Boy ', ' Boys ',' Girl',' Girls','Children',' Child','Infant','Teenager','Youngster','Youth', 'childhood']
s1 = [' Over 60 years old','Elderly']
s2 = [' Man ',' Men ',' Boy ',' Males ']
s3 = [' Woman ',' Women ']
s4 = [' Woman ',' Women ', 'Than males', 'Than men', 'Women are affected', 'Females are affected', 'Most often in females', 'and females', 'infected female', 'infected females']
s5 = ['Middle age']

In [17]:
def agebased_diseasegraph(para_age,sdg):
    if(para_age==0):
        sparl_age_gender_dislist(s,sdg)
    elif (para_age==1):
        
        sparl_age_gender_dislist(s5,sdg)
    elif (para_age==2):
        
        sparl_age_gender_dislist(s1,sdg)
    return "1"

In [18]:
def sexbased_diseasegraph(para_age,sdg):
    if(para_age==0):
        sparl_age_gender_dislist(s2,sdg)
    elif (para_age==1):
        sparl_age_gender_dislist(s4,sdg)
    return "1"

In [19]:
def causalr(q1,q2,g,a):
    sdg = Graph()
    sparl_signlist(q1,sdg)
    sparl_signmap(q1,sdg)
    sparl_aidiseaselist(q2,sdg)
    #dpbkg_generation()
    agebased_diseasegraph(a,sdg)
    #dpbkg_generation()
    sexbased_diseasegraph(g,sdg)
    dpbkg_generation(sdg)
    return sdg

In [34]:
testdf = pd.read_csv('testdata1.csv', encoding='utf-8')

In [22]:
def get_position_test(loci):
    listP = testdf['Position'][loci].split(',')
    listPr = []
    for j in range (0,len(listP)):
        if j==len(listP)-1 and listP[j].split('/')[-1][-1]==']':
            listPr.append(listP[j].split('/')[-1][0:-1])
        else:
            listPr.append(listP[j].split('/')[-1])
    return listPr

In [23]:
def get_sym_test(loci):
    listS = testdf['Symptom'][loci].split(',')
    listSr = []
    for j in range (0,len(listS)):
        if j==len(listS)-1 and listS[j].split('/')[-1][-1]==']':
            listSr.append(listS[j].split('/')[-1][0:-1])
        else:
            listSr.append(listS[j].split('/')[-1])
    return listSr

In [24]:
import time

In [44]:
acc_val = []
tri_siz = []
acc_tim = []

In [50]:
for i in range (130,len(testdf['Age'])):
    start = time.time()
    sp = get_sym_test(i)
    pp = get_position_test(i)
    if testdf['Age'][i]=='Yong': 
        ap = 0
    elif testdf['Age'][i]=='Elder':
        ap = 2
    else:
        ap = 1
    if testdf['Gen'][i]=='female':
        gp = 1
    else:
        gp = 0
    pdfs= pd.DataFrame(columns=['Disease','Probability'])
    os.remove('dpbkg1.ttl')
    cpdlgraph=causalr(sp, pp, gp, ap)
    outputr = ranking_result(pdfs)
    avflag = 0.0
    print testdf['Disease'][i]
    for k in range (0,len(outputr)):
        if outputr.index[k] == testdf['Disease'][i] and avflag ==0.00:
            print k
            if k<=1:
                avflag = 100.00
            elif 1<k<=5:
                avflag = 50.00
            elif k>5:
                avflag = 100.00/(k+1)
    end = time.time()
    acc_tim.append((end - start))
    acc_val.append(avflag) 
    tri_siz.append(len(cpdlgraph))
    print i,':',end - start
print acc_val
print tri_siz

http://ai.symptomchecking.org/condtion/Obesity
0
130 : 8.70519590378
http://ai.symptomchecking.org/condtion/Obstructive_sleep_apnoea
0
131 : 4.03303289413
http://ai.symptomchecking.org/condtion/Oesophageal_cancer
0
132 : 6.26655292511
http://ai.symptomchecking.org/condtion/Osteoarthritis
0
133 : 7.29630494118
http://ai.symptomchecking.org/condtion/Osteoporosis
0
134 : 3.54054903984
http://ai.symptomchecking.org/condtion/Osteosarcoma
1
135 : 2.33112597466
http://ai.symptomchecking.org/condtion/Otitis_externa
0
136 : 9.79984903336
http://ai.symptomchecking.org/condtion/Ovarian_cancer
0
137 : 15.6219291687
http://ai.symptomchecking.org/condtion/Overactive_thyroid
0
138 : 8.47855710983
http://ai.symptomchecking.org/condtion/Paget's_disease_of_the_nipple
2
139 : 2.62915110588
http://ai.symptomchecking.org/condtion/Pancreatic_cancer
140 : 4.07914400101
http://ai.symptomchecking.org/condtion/Parkinson's_disease
0
141 : 3.42290902138
http://ai.symptomchecking.org/condtion/Pelvic_organ_prolapse

In [57]:
tri_siz

[382,
 139,
 175,
 39,
 526,
 346,
 192,
 340,
 35,
 38,
 350,
 61,
 427,
 501,
 193,
 331,
 56,
 386,
 789,
 201,
 154,
 147,
 502,
 482,
 231,
 169,
 192,
 380,
 59,
 499,
 61,
 1175,
 310,
 497,
 604,
 34,
 134,
 14,
 73,
 124,
 102,
 551,
 296,
 252,
 124,
 173,
 24,
 37,
 444,
 139,
 138,
 435,
 202,
 258,
 625,
 175,
 278,
 156,
 833,
 53,
 47,
 229,
 223,
 396,
 337,
 617,
 213,
 295,
 180,
 60,
 108,
 396,
 530,
 91,
 28,
 135,
 308,
 0,
 127,
 24,
 64,
 101,
 70,
 50,
 196,
 517,
 289,
 210,
 417,
 54,
 307,
 403,
 429,
 144,
 69,
 207,
 67,
 268,
 28,
 256,
 124,
 5,
 541,
 25,
 516,
 681,
 65,
 256,
 10,
 141,
 46,
 773,
 254,
 22,
 230,
 294,
 618,
 472,
 575,
 132,
 265,
 253,
 163,
 816,
 309,
 51,
 193,
 68,
 50,
 243,
 602,
 140,
 218,
 428,
 184,
 55,
 646,
 1158,
 433,
 41,
 183,
 93,
 28,
 103,
 691,
 195,
 1164,
 1238,
 37,
 51,
 85,
 478,
 127,
 484,
 511,
 608,
 187,
 327,
 599,
 266,
 398,
 589,
 284,
 81,
 41,
 210,
 171,
 124,
 200,
 467,
 31,
 265,
 132,
 272,

In [58]:
sum(tri_siz)/len(tri_siz)

270

In [51]:
acc_tim

[7.215561151504517,
 4.2204060554504395,
 5.418489933013916,
 2.9893620014190674,
 8.546360969543457,
 5.7488110065460205,
 4.45470404624939,
 6.622989892959595,
 3.8709990978240967,
 2.8921267986297607,
 6.591385841369629,
 4.6728620529174805,
 6.335626125335693,
 9.295557022094727,
 4.55908203125,
 7.62446403503418,
 3.5169429779052734,
 6.293680906295776,
 10.96412181854248,
 4.8098649978637695,
 3.182835102081299,
 3.1397199630737305,
 8.39178204536438,
 9.68613600730896,
 7.584567070007324,
 4.498008966445923,
 3.690070152282715,
 6.261623859405518,
 5.376927137374878,
 7.203438997268677,
 1.9794421195983887,
 16.27248191833496,
 7.546174049377441,
 8.780511140823364,
 8.473816871643066,
 4.489439010620117,
 4.124191045761108,
 4.5901970863342285,
 2.720004081726074,
 3.1186890602111816,
 2.552304983139038,
 9.01206088066101,
 8.274058818817139,
 5.543776988983154,
 5.659477949142456,
 4.036626100540161,
 1.9944419860839844,
 2.3612821102142334,
 6.488200902938843,
 3.377362966537

In [56]:
sum(acc_tim)/len(acc_tim)

5.69548506581265

In [54]:
len(acc_val)

184

In [55]:
sum(acc_val)/len(acc_val)

91.54025322503584

209

In [158]:
acc_val1.count(100)

128

In [156]:
sum(acc_val1)/(len(acc_val1)-acc_val1.count(0))

98.20075757575758

In [229]:
def evaluation_test(loci):
    sp = get_sym_test(loci)
    pp = get_position_test(loci)
    if testdf['Age'][loci]=='Yong': 
        ap = 0
    elif testdf['Age'][loci]=='Elder':
        ap = 2
    else:
        ap = 1
    if testdf['Gen'][loci]=='female':
        gp = 1
    else:
        gp = 0
    print (sp)
    print (pp)
    print (ap)
    print (gp)
    pdfs= pd.DataFrame(columns=['Disease','Probability'])
    os.remove('dpbkg1.ttl')
    print ('oldttl has been moved')
    cpdlgraph=causalr(sp, pp, gp, ap)  

In [69]:
pdfs= pd.DataFrame(columns=['Disease','Probability'])
os.remove('dpbkg1.ttl')
print ('oldttl has been moved')
sp = ['Cough','Chest pain','Fever']
pp = ['Chest']
ap = 0
gp = 0
cpdlgraph=causalr(sp, pp, gp, ap)


oldttl has been moved


In [70]:
outputr = ranking_result(pdfs)

In [71]:
outputr

Disease
http://ai.symptomchecking.org/condtion/Acute_lymphoblastic_leukaemia_Children                      0.175160
http://ai.symptomchecking.org/condtion/Acute_lymphoblastic_leukaemia_Teenagers_and_young_adults    0.175160
http://ai.symptomchecking.org/condtion/Acute_lymphoblastic_leukaemia                               0.175160
http://ai.symptomchecking.org/condtion/Pneumonia                                                   0.175160
http://ai.symptomchecking.org/condtion/Lupus                                                       0.159535
http://ai.symptomchecking.org/condtion/Congenital_heart_disease                                    0.152778
http://ai.symptomchecking.org/condtion/Chest_pain                                                  0.152778
http://ai.symptomchecking.org/condtion/Measles                                                     0.147382
http://ai.symptomchecking.org/condtion/Whooping_cough                                              0.147382
http://ai.symptomche

In [252]:
evaluation2 = [None] * 3
v=0
for i in range (10,13):
    outputrk = evaluation_test(i)
    countg = Graph()
    countres = countg.parse('dpbkg1.ttl', format='turtle')
    evaluation2[v]=len(countres)
    v=v+1

[u'Clinical_urine_tests', u'Glucose', u'Adjective', u'Bleeding', u'Blood_test', u'Corticosteroid', u'Medical_imaging', u'Vomiting', u'Aldosterone']
[u'Hypothalamus']
2
1
oldttl has been moved
[u'Methanol', u'Metabolism', u'Recreational_drug_use', u'Sin', u'Euphoria']
[u'Ear']
0
1
oldttl has been moved
[u'Histamine', u'Food_allergy', u'Inflammation', u'Epinephrine', u'Corticosteroid', u'Food_intolerance', u'Skin_allergy_test', u'Antihistamine', u'Allergen_immunotherapy', u'Medical_history', u'Immune_system', u'Allergen', u'Foodborne_illness', u'Immunoglobulin_E', u'Pollen', u'Insect_bites_and_stings', u'Allergy']
[u'Basophil']
1
0
oldttl has been moved


In [251]:
outputrk = evaluation_test(10)
countg1 = Graph()
print len(countg1.parse('dpbkg1.ttl', format='turtle'))

[u'Clinical_urine_tests', u'Glucose', u'Adjective', u'Bleeding', u'Blood_test', u'Corticosteroid', u'Medical_imaging', u'Vomiting', u'Aldosterone']
[u'Hypothalamus']
2
1
oldttl has been moved
374


In [253]:
evaluation2

[374, 61, 396]

In [222]:
evaluation22.append(evaluation2)

In [223]:
evaluation22

[[587, 657, 989, 1391, 1573],
 [1696, 1738, 1764, 2225, 2397],
 [2398, 2417, 2539, 2801, 2853]]

In [99]:
testdf['Disease'][10]

u"http://ai.symptomchecking.org/condtion/Addison's_disease"